In [3]:
import sympy as sp 
import numpy as np

So, for simulating the ACF error, I need to know how to form the ACF in my case. I think it's given by:

R(dtau) = integral(x(t-tau)*x(t-tau_hat))dt 

Where tau_hat is the best estimate of the time delay 

In [21]:
import spreadingcode as sc
import math 

# create lookup table for prns  

def sample_sequence(prn_sequence, sampling_rate):
    # Determine the number of samples per chip
    samples_per_chip = int(sampling_rate / len(prn_sequence))

    # Repeat each element of the PRN sequence 'samples_per_chip' times
    sampled_sequence = np.repeat(prn_sequence, samples_per_chip)

    return sampled_sequence

# Assuming you already have the PRN sequence generated and stored in 'prn_sequence'
prn_sequence = sc.PRN(1)

# Sample the PRN sequence at 2 times the PRN chip rate
samples_per_chip = 10
sampling_rate = samples_per_chip * 1.023e3 # this is samples/chip * chips in 1 code period 
sampled_sequence = sample_sequence(prn_sequence, sampling_rate)

# test grabbing prn value at a certain continuous time 
tau = .9e-3 # s 
Tc = 1e-3 # ms
index = math.floor(tau/Tc * len(sampled_sequence))
x_tau = sampled_sequence[index] 
print(x_tau)
print(index)
print(len(sampled_sequence))

# get sequence at this delay 
# TODO I think they encode 1s to -1s and 0s to 1s 
x_tau_seq = np.roll(sampled_sequence, -1*index) # here, we're 9207 samples in, so make this the start and shift the earlier info to the end 
x_tau_seq[np.nonzero(x_tau_seq)] = -2
x_tau_adj = x_tau_seq + 1 

1
9207
10230


In [9]:
# simulate measurements
P = 130 # signal power -1*dB/Hz
n = 1 # inital noise 
r_r = 6360e3 # on earth's surface ECEF
v_r = 1  # m/s  
r_sv = 26560e3 # meters ECEF 

# iono and tropo
tau_I = 30* 60 # about a 30 minute time constant  
tau_T = 2*3600   # about a 2 hour time constant  
sig_I = 1.5 # should be about between 1-5 meters 
sig_T = .5 # should be rougly 1/2 of Iono 
I = 1
T = .5
Ts = (10*1.023e6)**-1 # Hz sample rate

c = 2.99792458e8; # speed of light in m/s 
L5 = 1176.45e6;   # L5 carrier frequency 
lam5 = c/L5;

CN0dBHz = -0;
CN0 = 10**(CN0dBHz/10);
R = 1/CN0;

dr = 5 # chips argument in error in position
dI = 1 # meters argument in error of inono offset
dT = .1 # meters argument in error of tropo offset
ddt_r = .001 # clock error in time in seconds
ddt_sv = 0 # clock error in time in seconds 

# measurement 
R = 1/c*(dr + dI + dT+ c*(ddt_r + ddt_sv))
y = (P**.5)*R*np.exp(-2*np.pi*1j/lam5 * (dr-dI+dT+c*(ddt_r + ddt_sv))) + np.random.normal(0, np.sqrt(R))

# propagate states 
I = I + Ts*(-1/tau_I * I + np.random.normal(0, sig_I)) # Ionospheric position offset 
T = T + Ts*(-1/tau_T * T + np.random.normal(0, sig_T)) # Tropospheric position offset 
b = b + Ts*(-1/tau_b * b + np.random.normal(0, sig_b))
v_r = b + np.random.normal(0, sig_v)
r_r = r_r + Ts*v_r
r = r_r - r_sv # r_r is the 3d user position, r_sv is 3d space vehicle position from ephemeris

print(R)

0.001000020347409807


In [ ]:
nominal_values = {
    'a': 1,
    'r_1': 2,
    'r_2': 3,
    'r_3': 4,
    'I': 5,
    'T': 6,
    'tr': 7,
}

# IMU Model 
# bias dynamic model (same in all three accelerometers)
tau_b = 3600 # time constant of bias random walk in seconds
w_a = 57**2 * 390 # velocity RW in [micro-g/sqrt(Hz)]^2 * bandwidth in Hz = variance in micro-g^2

# get IMU data
import scipy.io

# Load the .mat file
mat_data = scipy.io.loadmat('IMU_data_logan.mat')

# Print the keys (variable names from MATLAB)
print(mat_data.keys())

# Access a specific variable from the .mat file
#my_var = mat_data['variable_name_from_mat_file']
                                                                                                                                                                    

In [ ]:
import sympy as sp

# Matrix definitions
r_r = sp.Matrix([sp.symbols('r_1'), sp.symbols('r_2'), sp.symbols('r_3')])

# Remaining symbols
a, I, T, tr, tr_dot = sp.symbols('a I T tr tr_dot')

# Constants
c = 1
lambda_val = 1
const = {
     'tsv': .001,
     'r_sv': [100, 100, 100]
}

def compute_los(r, r_ref):
    """Compute the line of sight (LoS) vector given two positions."""
    delta_r = r - r_ref
    norm_delta_r = sp.sqrt(delta_r.dot(delta_r))
    r_los = delta_r / norm_delta_r
    return r_los, delta_r

def precompute_model():
    r_sv_matrix = sp.Matrix(const['r_sv'])
    r_los, delta_r = compute_los(r_r, r_sv_matrix)
    # Linearize the function wrt the state variables
    state_variables = [a, r_r[0], r_r[1], r_r[2], I, T, tr]
    
    nom_r = r_los.dot(delta_r) + I + T + c * (tr - const['tsv'])
    R = sp.Function('R')(1/c * (r_los.dot(delta_r) + I + T + c * tr - nom_r))
    h = a * R * sp.cos(2 * sp.pi / lambda_val * (r_los.dot(delta_r) - I + T + c * tr - nom_r))
    H_sym = h.jacobian(state_variables)
    
    # Convert to callable functions
    H_func = sp.lambdify((a, r_r, I, T, tr), H_sym, "numpy")
    h_func = sp.lambdify((a, r_r, I, T, tr), h, "numpy")
    
    return H_func, h_func

# Store the precomputed functions
H_func, h_func = precompute_model()

def m_model(nom, measurements):
    # Get the Jacobian matrix and h values
    H = H_func(nom['a'], [nom['r_1'], nom['r_2'], nom['r_3']], nom['I'], nom['T'], nom['tr'])
    h_val = h_func(nom['a'], [nom['r_1'], nom['r_2'], nom['r_3']], nom['I'], nom['T'], nom['tr'])
    
    y_star = measurements - h_val
    return H, y_star
